In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip3 uninstall keras-nightly
!pip3 uninstall -y tensorflow
!pip3 install keras==2.1.6
!pip3 install tensorflow==1.15.0
!pip3 install h5py==2.10.0

In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
%cd Mask_RCNN

In [ ]:
!python setup.py install
!pip install -r requirements.txt

In [ ]:
!pip install elementpath

In [ ]:
import elementpath
from google.colab import files
from os import listdir
from numpy import zeros, asarray, expand_dims, mean
from numpy import asarray
from mrcnn import visualize
from mrcnn.utils import Dataset, extract_bboxes, compute_ap
from mrcnn.config import Config
from mrcnn.visualize import display_instances
from mrcnn.model import MaskRCNN, load_image_gt, mold_image
import matplotlib.pyplot as pyplot
from matplotlib.patches import Rectangle, Arrow
import math
import cv2

In [ ]:
%cd /content/

In [ ]:
class PredictionConfig(Config):
  NAME = "manga_cfg"
  NUM_CLASSES = 1 + 3
  GPU_COUNT = 1
  IMAGES_PER_GPU = 1

In [ ]:
cfg = PredictionConfig()
model = MaskRCNN(mode="inference", model_dir="/content", config=cfg)

In [ ]:
model.load_weights("/dir/to/IS[Mask R-CNN]-3 (Manga109).h5", by_name=True)

In [ ]:
class_names = ['BG', 'face', 'text', 'frame']

In [ ]:
for i in range(20):
  image = cv2.imread('/dir/to/Validation/' + str(i+1) + '.jpg')
  results = model.detect([image], verbose=1)
  r = results[0]
  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

# Face-Text Association

In [ ]:
def arrow_face_text(image, cfg, model):
  results = model.detect([image], verbose=1)
  r = results[0]

  pyplot.figure(figsize=(10,10))
  pyplot.imshow(image)
  pyplot.title("Face to Text")

  ax = pyplot.gca()

  face_centers = []
  text_centers = []
  frame_corners = []
  count = 0

  for id in list(r['class_ids']):
    box = list(r['rois'])[count]
    count += 1

    y1, x1, y2, x2 = box
    width, height = x2 - x1, y2 - y1

    if id == 1:
      face_x = (x1 + x2)//2
      face_y = (y1 + y2)//2
      face_center = [face_x, face_y]
      face_centers.append(face_center)

      rect = Rectangle((x1, y1), width, height, fill=False, color="red")
      ax.add_patch(rect)

    elif id == 2:
      text_x = (x1 + x2)//2
      text_y = (y1 + y2)//2
      text_center = [text_x, text_y]
      text_centers.append(text_center)

      rect = Rectangle((x1, y1), width, height, fill=False, color="yellow")
      ax.add_patch(rect)

    elif id == 3:
      frame_corners.append([x1, x2, y1, y2])
      rect = Rectangle((x1, y1), width, height, fill=False, color="violet")
      ax.add_patch(rect)

  faces_to_texts = []

  for frame in frame_corners:
    x1, x2, y1, y2 = frame
    face_centers_filtered, text_centers_filtered = [], []
    num_faces, num_text = 0, 0

    for face in face_centers:
      if face[0] < x2 and face[0] > x1 and face[1] < y2 and face[1] > y1:
        num_faces += 1
        face_centers_filtered.append(face)

    for text in text_centers:
      if text[0] < x2 and text[0] > x1 and text[1] < y2 and text[1] > y1:
        num_text += 1
        text_centers_filtered.append(text)
    
    if num_faces >= num_text:
      for face in face_centers_filtered:
        if text_centers_filtered:
          nearest_text = text_centers_filtered[0]
          shortest_x = abs(face[0] - nearest_text[0])
          shortest_y = abs(face[1] - nearest_text[1])
          shortest_distance = math.sqrt(shortest_x**2 + shortest_y**2)

          for text in text_centers_filtered:
            distance_x = abs(face[0] - text[0])
            distance_y = abs(face[1] - text[1])
            distance = math.sqrt(distance_x**2 + distance_y**2)

            if distance < shortest_distance:
              shortest_distance = distance
              nearest_text = text

          face_to_text = [face, nearest_text]
          faces_to_texts.append(face_to_text)

    elif num_faces < num_text:
      for text in text_centers_filtered:
        if face_centers_filtered:
          nearest_face = face_centers_filtered[0]
          shortest_x = abs(text[0] - nearest_face[0])
          shortest_y = abs(text[1] - nearest_face[1])
          shortest_distance = math.sqrt(shortest_x**2 + shortest_y**2)

          for face in face_centers_filtered:
            distance_x = abs(face[0] - text[0])
            distance_y = abs(face[1] - text[1])
            distance = math.sqrt(distance_x**2 + distance_y**2)

            if distance < shortest_distance:
              shortest_distance = distance
              nearest_face = face

          face_to_text = [nearest_face, text]
          faces_to_texts.append(face_to_text)

  for face_to_text in faces_to_texts:
    face, text = face_to_text

    face_x, face_y = face
    text_x, text_y = text

    length_x = abs(face_x - text_x)
    length_y = abs(face_y - text_y)

    if face_x > text_x: #face is to the right of text
      length_x *= -1
      
    if face_y > text_y: #face is below text
      length_y *= -1

    arrow = Arrow(face_x, face_y, length_x, length_y, color="cornflowerblue")
    ax.add_patch(arrow)

  pyplot.show()

In [ ]:
for i in range(20):
  image = cv2.imread('/dir/to/Validation/' + str(i+1) + '.jpg')
  width = image.shape[1]
  arrow_face_text(image, cfg, model)

# Text Order Determination

In [ ]:
def order_frame_halves(unordered_frames):
  ordered_frames = []
  while unordered_frames:
    for frame in unordered_frames:
      for other_frame in unordered_frames:
        if other_frame[3] < frame[1]: # if there is a frame above the current frame
          break # the current frame is not the next frame to read
      else:
        ordered_frames.append(frame[:-1]) # the current frame is the next frame
        unordered_frames.remove(frame)

  return ordered_frames

In [ ]:
def order_text(image, img_width, cfg, model):
  results = model.detect([image], verbose=1)
  r = results[0]

  pyplot.figure(figsize=(10,10))
  pyplot.imshow(image)
  pyplot.title("Order Text")

  ax = pyplot.gca()

  text_centers = []
  total_unordered_frames = []
  count = 0

  for id in list(r['class_ids']):
    box = list(r['rois'])[count]
    count += 1

    y1, x1, y2, x2 = box
    width, height = x2 - x1, y2 - y1

    if id == 2:
      text_x = (x1 + x2)//2
      text_y = (y1 + y2)//2
      text_center = [text_x, text_y]
      text_centers.append(text_center)

      rect = Rectangle((x1, y1), width, height, fill=False, color="cornflowerblue")
      ax.add_patch(rect)
    elif id == 3:
      frame_x_center = (x1 + x2)//2
      total_unordered_frames.append([x1, y1, x2, y2, frame_x_center]) # the corners are ordered this way to improve sorting later on

  total_unordered_frames.sort(reverse=True)
  ordered_frames = [] # 1st, 2nd, 3rd, ...

  half_line = img_width//2

  unordered_frames_first = [frame for frame in total_unordered_frames if frame[-1] > half_line] # frames to the right of half line
  unordered_frames_second = [frame for frame in total_unordered_frames if frame[-1] <= half_line] # frames to the left of half line

  ordered_frames_first = order_frame_halves(unordered_frames_first)
  ordered_frames_second = order_frame_halves(unordered_frames_second)

  index = 1
  
  for frame in ordered_frames_first:
    x1, y1, x2, y2 = frame
    text_centers_filtered = []

    for text in text_centers:
      if text[0] < x2 and text[0] > x1 and text[1] < y2 and text[1] > y1:
        text_centers_filtered.append([text[0], -text[1]])
    
    text_centers_filtered.sort(reverse=True)
    for text in text_centers_filtered:
      pyplot.text(text[0], -text[1], index, color="red")
      index += 1

  for frame in ordered_frames_second:
    x1, y1, x2, y2 = frame
    text_centers_filtered = []

    for text in text_centers:
      if text[0] < x2 and text[0] > x1 and text[1] < y2 and text[1] > y1:
        text_centers_filtered.append([text[0], -text[1]])
    
    text_centers_filtered.sort(reverse=True)
    for text in text_centers_filtered:
      pyplot.text(text[0], -text[1], index, color="red")
      index += 1

  pyplot.show()

In [ ]:
for i in range(20):
  image = cv2.imread('/dir/to/Validation/' + str(i+1) + '.jpg')
  width = image.shape[1]
  order_text(image, width, cfg, model)